# Recursive QAOA

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import networkx as nx
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import json

from openqaoa.qaoa_parameters import QAOACircuitParams, create_qaoa_variational_params, Hamiltonian
from openqaoa.devices import create_device
from openqaoa.backends.qaoa_backend import get_qaoa_backend
from openqaoa.optimizers import get_optimizer
from openqaoa.utilities import *
from openqaoa.workflows.optimizer import QAOA, RQAOA
from openqaoa.problems.problem import QUBO, MaximumCut, MinimumVertexCover

In [ ]:
# Set plotting style
plt.rcParams["figure.figsize"] = (15,10)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 600
#plt.rcParams['font.size'] = '12'
#plt.rc('font', family='serif', size=600)

sns.axes_style("whitegrid")
sns.set_context("paper", font_scale=2.5)

In [ ]:
def random_k_regular_graph_normal_distribution(degree: int,
                           nodes: List[int],
                           seed: int = None,
                           weighted: bool = False,
                           sigma: float = 1.0,
                           mu: float = 0.0,
                           biases: bool = False) -> nx.Graph:
    """
    Produces a random graph with specified number of nodes, each having degree k.

    Parameters
    ----------
    degree: `int`
        Desired degree for the nodes.
    nodes: `list`
        The node set of the graph.
    seed: `int`, optional
        A seed for the random number generator.
    weighted: `bool`, optional
        Whether the edge weights should be uniform or different. If false, all weights are set to 1.
        If true, the weight is set to a random number drawn from the uniform distribution in the
        interval 0 to 1.
    biases: `bool`, optional
        Whether or not the graph nodes should be assigned a weight.
        If true, the weight is set to a random number drawn from the uniform
        distribution in the interval 0 to 1.

    Returns
    -------
    nx.Graph: `Networkx Graph`
        A graph with the properties as specified.
    """
    # Set numpy seed
    np.random.seed(seed=seed)

    # Create a random regular graph on the nodes
    G = nx.random_regular_graph(degree, len(nodes), seed)

    # Relabel nodes
    nx.relabel_nodes(G, {i: n for i, n in enumerate(nodes)})

    # Add edges between nodes
    for edge in G.edges():

        # If weighted attribute is False, all weights are set to 1
        if not weighted:
            G[edge[0]][edge[1]]['weight'] = 1

        # If weighted attribute is True, weights are assigned as random integers
        else:
            G[edge[0]][edge[1]]['weight'] = np.random.normal(mu, sigma)

    # If biases attribute is True, add node weights as random integers
    if biases:
        for node in G.nodes():
            G.nodes[node]['weight'] = np.random.rand()

    return G

In [ ]:
def run_Rqaoa_experiment(qubo, optimizer, init_beta, init_gamma):  
    
    # Define the RQAOA object (default rqaoa_type = 'adaptive')
    R = RQAOA()

    # Set parameters for RQAOA, in this case we fix the n_max to 1 (default), the final cutoff value to 3
    R.set_rqaoa_parameters(steps = 1, n_cutoff=3)

    ## Setting up the QAOA properties

    # Set the properties you want - These values are actually the default ones!
    #R.set_circuit_properties(p=1, param_type='standard', init_type='ramp', mixer_hamiltonian='x')
    R.set_circuit_properties(p=1, init_type='custom', variational_params_dict={"betas":[init_beta], "gammas":[init_gamma]}, mixer_hamiltonian='x')

    # Define the device you want to run your problem on using the create_device() function - Here we choose the local wavefunction simulator supported by OpenQAOA
    device = create_device(location='local', name='vectorized')
    R.set_device(device)

    # Set the classical method used to optimiza over QAOA angles and its properties
    R.set_classical_optimizer(method=optimizer, 
                              optimizer_options = {"stepsize" : 0.01},
                              tol=10**(-6), maxfev=6000, maxiter=2000, 
                              jac='finite_difference',
                              #parameter shift
                              hess='finite_difference',
                              optimization_progress=True, 
                              cost_progress=True, parameter_log=True)

    # Here are other ways in which the above selection can be generalized!

    # device = create_device(location = 'qcs',name = 'Aspen-11', as_qvm = True, client_configuraton = qcs_credentials)
    # r.set_device(device)
    # r.set_backend_properties(n_shots=100, cvar_alpha=1)
    # r.set_circuit_properties(param_type = 'extended', init_type = 'fourier_extended')
    # r.set_classical_optimizer(method='Nelder-Mead', maxiter=1000)
    

    # Compile problem instance on RQAOA, just like with QAOA
    R.compile(maxcut_qubo)
    
    R.optimize()
    
    return R.results


In [ ]:
# helper function to produce the cost landscape
def create_cost_landscape_std(hamiltonian, betas, gammas):
    """
    This function constructs a 2-D array containing cost values for different pairs
    of parameter values.
    
    """
    
    cost_landscape = np.zeros((betas.size, gammas.size), dtype=float)
    
    for i, beta in enumerate(betas):
        for j, gamma in enumerate(gammas):
            variational_params = [beta, gamma]
            cost_landscape[i,j] = energy_expectation_analytical(variational_params, hamiltonian)
            
           
    return cost_landscape

In [ ]:
def plot_landscape(cost_landscape, betas, gammas, sigma, mu):

    plt.contourf(cost_landscape, cmap='viridis', levels=100, extent=(gammas[0],gammas[-1],betas[0],betas[-1]))

    #plt.title(r'Cost Landscape for Standard Parameterization for $\sigma$={}'.format(sigma))
    plt.xlabel(r'$\gamma$')
    plt.ylabel(r'$\beta$')
    plt.colorbar()
    #plt.savefig("cost_landscape_sigma={}_mu={}.png".format(sigma, mu))
    plt.show()
    

In [ ]:
def plot_landscape_with_opt_trajectory(cost_landscape, optimization_angles, betas, gammas, optimizer):
    plt.contourf(cost_landscape, cmap='viridis', levels=100, extent=(gammas[0],gammas[-1],betas[0],betas[-1]))
        
    # TODO not sure about the order here!
    plt.plot(optimization_angles[:, 1], optimization_angles[:, 0], '--', color='black',  )  
    plt.plot(optimization_angles[0, 1], optimization_angles[0, 0], 'x', color='black',  ) 
    
    plt.plot(optimization_angles[-1, 1], optimization_angles[-1, 0], 'x', color='orange',  )

    #plt.title(r'Cost Landscape for Standard Parameterization for $\sigma$={}'.format(sigma))
    plt.xlabel(r'$\gamma$')
    plt.ylabel(r'$\beta$')
    plt.colorbar()
    plt.savefig("cost_landscape_trajectory_optimizer={}.png".format(optimizer))
    plt.show()

In [ ]:
# Define problem

n = 10
k = 3
sigma_list = [0.1, 1, 10, 100]
mu = 5.0 
betas = np.linspace(-np.pi/2, np.pi/2, 50)
gammas = np.linspace(-np.pi, np.pi, 50)


optimizer_list = ["nelder-mead"]
sigma_list = [10]

#seed_list = [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]

seed_list = [1]

data = []  # list to contain optimization results to be converted into a data frame

for sigma in sigma_list:
    print("Sigma =", sigma)
    
    for seed in seed_list:
        print("Seed =", seed)

        # Define 3wR graph and plot it
        g = random_k_regular_graph_normal_distribution(degree=k, nodes=range(n), seed=seed, weighted=True, mu=mu, sigma=sigma, biases=False)
        plot_graph(g)
        #plt.savefig("graph_sigma={}.png".format(sigma))
        #plt.clf()

        # Define te problem and translate it into a binary Qubo.
        maxcut_prob = MaximumCut(g)
        maxcut_qubo = maxcut_prob.get_qubo_problem()

        # Define Hamiltonian
        cost_hamiltonian = hamiltonian_from_graph(g)

        #cost_landscape_std = create_cost_landscape_std(cost_hamiltonian, betas, gammas)
        #plot_landscape(cost_landscape_std, betas, gammas, sigma, mu)


        for optimizer in optimizer_list:
            print("Optimizer =", optimizer)

            # initialize away from the minimum
            init_beta = 0.5
            init_gamma = -0.2

            opt_results = run_Rqaoa_experiment(maxcut_qubo, optimizer, init_beta, init_gamma)

            optimization_energy = list(opt_results['solution'].values())[0]
            
            # Solve the problem analytically
            energy, configuration = ground_state_hamiltonian(cost_hamiltonian)

            # Obtain approximation ratio
            r = np.abs(optimization_energy) / np.abs(energy)
            
            data.append([sigma, optimizer, seed, r])

df = pd.DataFrame(np.array(data), columns=['sigma', 'optimizer', 'seed', 'r'])

df['sigma'] = df['sigma'].astype(float)
df['optimizer'] = df['optimizer'].astype(str)
df['seed'] = df['seed'].astype(int)
df['r'] = df['r'].astype(float)

In [ ]:
opt_results

In [ ]:
intermediate_steps = len(opt_results['intermediate_steps'])
intermediate_steps

In [ ]:
for i in range(0, intermediate_steps):
    intermediate_qubo = opt_results['intermediate_steps'][i]['QUBO']
    intermediate_hamiltonian = intermediate_qubo.hamiltonian
    intermediate_g = graph_from_hamiltonian(intermediate_hamiltonian)
    print(intermediate_g)
    plot_graph(intermediate_g)

In [ ]:
opt_results['intermediate_exp_vals_z']

In [ ]:
plt.imshow(opt_results['intermediate_corr_matrix'][0], cmap="cool")
plt.colorbar()

In [ ]:
plt.imshow(opt_results['intermediate_corr_matrix'][1], cmap="cool")
plt.colorbar()

In [ ]:
for i in range(0, intermediate_steps):
    intermediate_qubo = opt_results['intermediate_steps'][i]['QUBO']
    intermediate_hamiltonian = intermediate_qubo.hamiltonian
    intermediate_g = graph_from_hamiltonian(intermediate_hamiltonian)
    print(intermediate_g)
    plot_graph(intermediate_g)
    plt.clf()
    cost_landscape = create_cost_landscape_std(intermediate_hamiltonian, betas, gammas)
    plot_landscape(cost_landscape, betas, gammas, sigma, mu)


In [ ]:
intermediate_qaoa = opt_results['intermediate_steps'][1]['QAOA']
intermediate_qaoa.results.plot_cost()

### Averaging over Optimizers

In [ ]:
# Define problem

n = 12
k = 3
sigma_list = [0.1, 1, 10, 100]
mu = 5.0 
betas = np.linspace(-np.pi/2, np.pi/2, 50)
gammas = np.linspace(-np.pi, np.pi, 50)


#optimizer_list = ["vgd", "rmsprop", 'newton-cg', 'cobyla', 'nelder-mead', 'powell' ]

optimizer_list = ["rmsprop", 'newton-cg', 'cobyla', 'nelder-mead']

seed_list = [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]

#data = []  # list to contain optimization results to be converted into a data frame

for sigma in sigma_list:
    print("Sigma =", sigma)
    
    for seed in seed_list:
        print("Seed =", seed)

        # Define 3wR graph and plot it
        g = random_k_regular_graph_normal_distribution(degree=k, nodes=range(n), seed=seed, weighted=True, mu=mu, sigma=sigma, biases=False)
        #plot_graph(g)
        #plt.savefig("graph_sigma={}.png".format(sigma))
        #plt.clf()

        # Define te problem and translate it into a binary Qubo.
        maxcut_prob = MaximumCut(g)
        maxcut_qubo = maxcut_prob.get_qubo_problem()

        # Define Hamiltonian
        cost_hamiltonian = hamiltonian_from_graph(g)

        #cost_landscape_std = create_cost_landscape_std(cost_hamiltonian, betas, gammas)
        #plot_landscape(cost_landscape_std, betas, gammas, sigma, mu)


        for optimizer in optimizer_list:
            print("Optimizer =", optimizer)

            opt_results = run_Rqaoa_experiment(maxcut_qubo, optimizer, init_beta, init_gamma)  # be very careful here which initialization is given inside the function

            optimization_energy = list(opt_results['solution'].values())[0]
            
            # Solve the problem analytically
            energy, configuration = ground_state_hamiltonian(cost_hamiltonian)

            # Obtain approximation ratio
            r = np.abs(optimization_energy) / np.abs(energy)
            
            data.append([sigma, optimizer, seed, r])

df = pd.DataFrame(np.array(data), columns=['sigma', 'optimizer', 'seed', 'r'])

df['sigma'] = df['sigma'].astype(float)
df['optimizer'] = df['optimizer'].astype(str)
df['seed'] = df['seed'].astype(int)
df['r'] = df['r'].astype(float)

In [ ]:
df.to_csv("data_23.11_RQAOA_seeds_100_ramp_optimizers.csv")

In [ ]:
#df = pd.read_csv('data_22.11_RQAOA_seeds_100_optimizers.csv', index_col=0)

In [ ]:
df

In [ ]:
to_be_removed = df.loc[df.optimizer == 'powell', ['sigma', 'seed', 'r']]

In [ ]:
bad_indexes = df.reset_index().merge(to_be_removed)['index']

In [ ]:
df = df.loc[~df.index.isin(bad_indexes)]

In [ ]:
#plt.xscale('log')
plt.ylim(0, 1)
box_plot = sns.boxplot(data = df, x = 'sigma', y ='r', hue="optimizer")

plt.savefig("Approximation_ratio_mu=5.0_RQAOA_ramp_optimizers_NEW.png")


In [ ]:
plt.xscale('log')
plt.ylim(0, 1)
sns.scatterplot(data = df, x = 'sigma', y ='r', hue="optimizer")

plt.savefig("Approximation_ratio_mu=5.0_RQAOA_ramp_optimizers_scatterplot_NEW.png")


### Time the perfomance of each optimizer

In [ ]:
%%time

# Define problem

n = 12
k = 3
sigma_list = [0.1, 1, 10, 100]
mu = 5.0 
betas = np.linspace(-np.pi/2, np.pi/2, 50)
gammas = np.linspace(-np.pi, np.pi, 50)


#optimizer_list = ["vgd", "rmsprop", 'newton-cg', 'cobyla', 'nelder-mead', 'powell' ]

optimizer_list = ["rmsprop"]

#seed_list = [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]

seed_list = [26]
sigma_list = [1]

#data = []  # list to contain optimization results to be converted into a data frame

for sigma in sigma_list:
    print("Sigma =", sigma)
    
    for seed in seed_list:
        print("Seed =", seed)

        # Define 3wR graph and plot it
        g = random_k_regular_graph_normal_distribution(degree=k, nodes=range(n), seed=seed, weighted=True, mu=mu, sigma=sigma, biases=False)
        #plot_graph(g)
        #plt.savefig("graph_sigma={}.png".format(sigma))
        #plt.clf()

        # Define te problem and translate it into a binary Qubo.
        maxcut_prob = MaximumCut(g)
        maxcut_qubo = maxcut_prob.get_qubo_problem()

        # Define Hamiltonian
        cost_hamiltonian = hamiltonian_from_graph(g)

        #cost_landscape_std = create_cost_landscape_std(cost_hamiltonian, betas, gammas)
        #plot_landscape(cost_landscape_std, betas, gammas, sigma, mu)


        for optimizer in optimizer_list:
            print("Optimizer =", optimizer)


            opt_results = run_Rqaoa_experiment(maxcut_qubo, optimizer, init_beta, init_gamma)

            optimization_energy = list(opt_results['solution'].values())[0]
            
            # Solve the problem analytically
            energy, configuration = ground_state_hamiltonian(cost_hamiltonian)

            # Obtain approximation ratio
            r = np.abs(optimization_energy) / np.abs(energy)
            
            data.append([sigma, optimizer, seed, r])
            

'''
df = pd.DataFrame(np.array(data), columns=['sigma', 'optimizer', 'seed', 'r'])

df['sigma'] = df['sigma'].astype(float)
df['optimizer'] = df['optimizer'].astype(str)
df['seed'] = df['seed'].astype(int)
df['r'] = df['r'].astype(float)
'''

In [ ]:
data_time=[]

data_time.append(['rmsprop', 72.0])
data_time.append(['cobyla', 15.3])
data_time.append(['powell', 1.04])
data_time.append(['nelder-mead', 1.02])
print(data_time)
df_time = pd.DataFrame(np.array(data_time), columns=['optimizer', 'time'])


df_time['optimizer'] = df_time['optimizer'].astype(str)
df_time['time'] = df_time['time'].astype(float)  # in seconds

In [ ]:
df_time

### Test why rmsprop does so bad for sigma=0.1

In [ ]:

# Define problem

n = 12
k = 3
sigma_list = [0.1, 1, 10, 100]
mu = 5.0 
betas = np.linspace(-np.pi/2, np.pi/2, 50)
gammas = np.linspace(-np.pi, np.pi, 50)


#optimizer_list = ["vgd", "rmsprop", 'newton-cg', 'cobyla', 'nelder-mead', 'powell' ]

optimizer_list = ["cobyla"]

#seed_list = [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]

seed_list = [9]
sigma_list = [0.1]

#data = []  # list to contain optimization results to be converted into a data frame

for sigma in sigma_list:
    print("Sigma =", sigma)
    
    for seed in seed_list:
        print("Seed =", seed)

        # Define 3wR graph and plot it
        g = random_k_regular_graph_normal_distribution(degree=k, nodes=range(n), seed=seed, weighted=True, mu=mu, sigma=sigma, biases=False)
        #plot_graph(g)
        #plt.savefig("graph_sigma={}.png".format(sigma))
        #plt.clf()

        # Define te problem and translate it into a binary Qubo.
        maxcut_prob = MaximumCut(g)
        maxcut_qubo = maxcut_prob.get_qubo_problem()

        # Define Hamiltonian
        cost_hamiltonian = hamiltonian_from_graph(g)

        #cost_landscape_std = create_cost_landscape_std(cost_hamiltonian, betas, gammas)
        #plot_landscape(cost_landscape_std, betas, gammas, sigma, mu)


        for optimizer in optimizer_list:
            print("Optimizer =", optimizer)
            
            # ramp initialization
            init_beta = 0.35
            init_gamma = 0.35


            opt_results = run_Rqaoa_experiment(maxcut_qubo, optimizer, init_beta, init_gamma)

            optimization_energy = list(opt_results['solution'].values())[0]
            
            # Solve the problem analytically
            energy, configuration = ground_state_hamiltonian(cost_hamiltonian)

            # Obtain approximation ratio
            r = np.abs(optimization_energy) / np.abs(energy)
            
            #data.append([sigma, optimizer, seed, r])
            

'''
df = pd.DataFrame(np.array(data), columns=['sigma', 'optimizer', 'seed', 'r'])

df['sigma'] = df['sigma'].astype(float)
df['optimizer'] = df['optimizer'].astype(str)
df['seed'] = df['seed'].astype(int)
df['r'] = df['r'].astype(float)
'''

In [ ]:
opt_results

In [ ]:
opt_results.dump('rmsprop_fails_cobyla.json')

In [ ]:
intermediate_steps = len(opt_results['intermediate_steps'])
intermediate_steps

In [ ]:
for i in range(0, intermediate_steps):
    intermediate_qubo = opt_results['intermediate_steps'][i]['QUBO']
    intermediate_hamiltonian = intermediate_qubo.hamiltonian
    intermediate_g = graph_from_hamiltonian(intermediate_hamiltonian)
    print(intermediate_g)
    plot_graph(intermediate_g)

In [ ]:
opt_results['intermediate_steps'][0]['QAOA'].results.intermediate

In [ ]:
for i in range(0, intermediate_steps):
    intermediate_qubo = opt_results['intermediate_steps'][i]['QUBO']
    intermediate_hamiltonian = intermediate_qubo.hamiltonian
    intermediate_g = graph_from_hamiltonian(intermediate_hamiltonian)
    print(intermediate_g)
    plot_graph(intermediate_g)
    plt.clf()
    cost_landscape = create_cost_landscape_std(intermediate_hamiltonian, betas, gammas)
    plot_landscape(cost_landscape, betas, gammas, sigma, mu)


In [ ]:
# Opening JSON file
f = open('rmsprop_fails.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
optimization_angles = np.array(data['intermediate_steps'][3]['QAOA']['intermediate']['angles log'])

intermediate_hamiltonian = opt_results['intermediate_steps'][3]['QUBO'].hamiltonian
cost_landscape = create_cost_landscape_std(intermediate_hamiltonian, betas, gammas)
  
# Closing file
f.close()

In [ ]:
plot_landscape_with_opt_trajectory(cost_landscape, optimization_angles, betas, gammas, optimizer)

In [ ]:
# Opening JSON file
f = open('rmsprop_fails_cobyla.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
for i in [0,1,2,3]:
    # Iterating through the json
    # list
    optimization_angles = np.array(data['intermediate_steps'][i]['QAOA']['intermediate']['angles log'])
    intermediate_hamiltonian = opt_results['intermediate_steps'][i]['QUBO'].hamiltonian
    cost_landscape = create_cost_landscape_std(intermediate_hamiltonian, betas, gammas)



    plot_landscape_with_opt_trajectory(cost_landscape, optimization_angles, betas, gammas, optimizer)

# Closing file
f.close()

In [ ]:
int_qaoa = opt_results['intermediate_steps'][i]['QAOA']


In [ ]:
opt_results

## Averaging over initializations

In [ ]:
# Define problem

n = 12
k = 3
sigma_list = [0.1]
mu = 5.0 
betas = np.linspace(-np.pi/2, np.pi/2, 50)
gammas = np.linspace(-np.pi, np.pi, 50)


#optimizer_list = ["vgd", "rmsprop", 'newton-cg', 'cobyla', 'nelder-mead', 'powell' ]

optimizer_list = ['cobyla']

#seed_list = [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]

seed_list = [1, 2]

#data = []  # list to contain optimization results to be converted into a data frame

for sigma in sigma_list:
    print("Sigma =", sigma)
    
    for seed in seed_list:
        print("graph_seed=", seed)

        # Define 3wR graph and plot it
        g = random_k_regular_graph_normal_distribution(degree=k, nodes=range(n), seed=seed, weighted=True, mu=mu, sigma=sigma, biases=False)
        #plot_graph(g)
        #plt.savefig("graph_sigma={}.png".format(sigma))
        #plt.clf()

        # Define te problem and translate it into a binary Qubo.
        maxcut_prob = MaximumCut(g)
        maxcut_qubo = maxcut_prob.get_qubo_problem()

        # Define Hamiltonian
        cost_hamiltonian = hamiltonian_from_graph(g)

        #cost_landscape_std = create_cost_landscape_std(cost_hamiltonian, betas, gammas)
        #plot_landscape(cost_landscape_std, betas, gammas, sigma, mu)
        
        # Solve the problem analytically
        energy, configuration = ground_state_hamiltonian(cost_hamiltonian)

        for optimizer in optimizer_list:
            print("Optimizer =", optimizer)
            
            r_best = 0.5
            r_all = []

            # initialize randomly
            for init_seed in [10, 20, 30]:
                random.seed(init_seed)
                init_beta = random.uniform(0, np.pi/2)
                init_gamma = random.uniform(0, 5*np.pi)
                
                print(init_beta)
                print(init_gamma)

                opt_results = run_Rqaoa_experiment(maxcut_qubo, optimizer, init_beta, init_gamma)
                
                

                optimization_energy = list(opt_results['solution'].values())[0]

                # Obtain approximation ratio
                r = np.abs(optimization_energy) / np.abs(energy)
                r_all.append()
                
                if r > r_best:
                    r_best = r
                    
            r_avr = np.average(r_all)

            data.append([sigma, optimizer, seed, r_best, r_avr])

df = pd.DataFrame(np.array(data), columns=['sigma', 'optimizer', 'seed', 'r_best', 'r_avr'])

df['sigma'] = df['sigma'].astype(float)
df['optimizer'] = df['optimizer'].astype(str)
df['seed'] = df['seed'].astype(int)
df['r'] = df['r'].astype(float)